In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import os
# os.environ["KERAS_BACKEND"] = "torch"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import pandas as pd

from skforecast.deep_learning import ForecasterRnn
from skforecast.deep_learning.utils import create_and_compile_model

# set backend torch
import torch
import keras
print(keras.__version__)
print(keras.backend.backend())



3.6.0
tensorflow


# Test with Ximo

In [3]:
from skforecast.datasets import fetch_dataset
data = fetch_dataset('bike_sharing_extended_features')

end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val = data.loc[end_train:end_validation, :]
data_test = data.loc[end_validation:, :]


bike_sharing_extended_features
------------------------------
Hourly usage of the bike share system in the city of Washington D.C. during the
years 2011 and 2012. In addition to the number of users per hour, the dataset
was enriched by introducing supplementary features. Addition includes calendar-
based variables (day of the week, hour of the day, month, etc.), indicators for
sunlight, incorporation of rolling temperature averages, and the creation of
polynomial features generated from variable pairs. All cyclic variables are
encoded using sine and cosine functions to ensure accurate representation.
Fanaee-T,Hadi. (2013). Bike Sharing Dataset. UCI Machine Learning Repository.
https://doi.org/10.24432/C5W894.
Shape of the dataset: (17352, 90)


# Without Exog

In [31]:
lags = 24
steps = 36
levels = ["users"]
activation = "relu"
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]

model = create_and_compile_model(
    series=data[levels],
    lags=lags,
    steps=steps,
    levels=levels,
    # exog=data[exog_features],
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    optimizer=optimizer,
    loss=loss,
)

model.summary()

# ==============================================================================

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    fit_kwargs={
        "epochs": 2,  # Number of epochs to train the model.
        "batch_size": 2048,  # Batch size to train the model.
    },
)    


# Backtesting model with exogenous variables on test data 
# # ============================================================================== 
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster_multiseries
 
cv = TimeSeriesFold( steps = 36, initial_train_size = len(data[:end_validation]), refit = False, allow_incomplete_fold=False)
metric, predictions = backtesting_forecaster_multiseries(
    forecaster = forecaster, 
    series = data[levels], 
    # exog = data[exog_features], 
    cv = cv, 
    metric = 'mean_absolute_error'
    )

metric

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ series_input (InputLayer)       │ (None, 24, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector_2 (RepeatVector)  │ (None, 36, 100)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 36, 128)        │        12,928 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 36, 64)         │         8,256 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 36, 1)          │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,049 (242.38 KB)

 Trainable params: 62,049 (242.38 KB)

 Non-trainable params: 0 (0.00 B)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Using device: cuda
Epoch 1/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 596ms/step - loss: 0.0469
Epoch 2/2
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 581ms/step - loss: 0.0323


  0%|          | 0/80 [00:00<?, ?it/s]

,levels,mean_absolute_error
0,users,169.796026


In [32]:
forecaster.fit(series = data[levels])

Using device: cuda
Epoch 1/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 629ms/step - loss: 0.0549
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 822ms/step - loss: 0.0356


In [33]:
# print(forecaster.keras_backend)
print(forecaster.X_train_dim_names_)
print(forecaster.y_train_dim_names_)
print(forecaster.series_names_in_)
print(forecaster.exog_in_)

{0: DatetimeIndex(['2011-01-09 00:00:00', '2011-01-09 01:00:00',
               '2011-01-09 02:00:00', '2011-01-09 03:00:00',
               '2011-01-09 04:00:00', '2011-01-09 05:00:00',
               '2011-01-09 06:00:00', '2011-01-09 07:00:00',
               '2011-01-09 08:00:00', '2011-01-09 09:00:00',
               ...
               '2012-12-29 03:00:00', '2012-12-29 04:00:00',
               '2012-12-29 05:00:00', '2012-12-29 06:00:00',
               '2012-12-29 07:00:00', '2012-12-29 08:00:00',
               '2012-12-29 09:00:00', '2012-12-29 10:00:00',
               '2012-12-29 11:00:00', '2012-12-29 12:00:00'],
              dtype='datetime64[ns]', name='date_time', length=17293, freq='h'), 1: ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11', 'lag_12', 'lag_13', 'lag_14', 'lag_15', 'lag_16', 'lag_17', 'lag_18', 'lag_19', 'lag_20', 'lag_21', 'lag_22', 'lag_23', 'lag_24'], 2: ['users']}
{0: DatetimeIndex(['2011-01-09 00:0

In [34]:
forecaster.X_train_dim_names_[1]

['lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10',
 'lag_11',
 'lag_12',
 'lag_13',
 'lag_14',
 'lag_15',
 'lag_16',
 'lag_17',
 'lag_18',
 'lag_19',
 'lag_20',
 'lag_21',
 'lag_22',
 'lag_23',
 'lag_24']

In [35]:
forecaster.y_train_dim_names_[2]

['users']

In [36]:
forecaster._create_predict_inputs()

(array([[[0.04098361],
         [0.02766393],
         [0.01844262],
         [0.01434426],
         [0.00614754],
         [0.00102459],
         [0.00717213],
         [0.01229508],
         [0.03278689],
         [0.07479508],
         [0.12397541],
         [0.13831967],
         [0.14651639],
         [0.17213115],
         [0.16290984],
         [0.14036885],
         [0.1352459 ],
         [0.125     ],
         [0.12704918],
         [0.10348361],
         [0.0727459 ],
         [0.04713115],
         [0.03586066],
         [0.04918033]]]),
 [],
 [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36],
 ['users'],
 DatetimeIndex(['2012-12-31 00:00:00', '2012-12-31 01:00:00',
                '2012-12-31 02:00:00', '2012-12-31 03:00:00',
                '2012-12-31 04:00:00', '2012-12-31 05:00:00',
                '2012-12-31 06:00:

# With Exog Features

In [18]:
exog_features = [ 'month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'holiday_previous_day', 'holiday_next_day', 'temp_roll_mean_1_day', 'temp_roll_mean_7_day', 'temp_roll_max_1_day', 'temp_roll_min_1_day', 'temp_roll_max_7_day', 'temp_roll_min_7_day', 'temp', 'holiday' ]

In [19]:
lags = 24
steps = 36
levels = ["users"]
activation = "relu"
# optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss = keras.losses.MeanSquaredError()
recurrent_units = 100
dense_units = [128, 64]

model = create_and_compile_model(
    series=data[levels],
    lags=lags,
    steps=steps,
    levels=levels,
    exog=data[exog_features],
    recurrent_units=recurrent_units,
    dense_units=dense_units,
    activation=activation,
    # optimizer=optimizer,
    loss=loss,
)

model.summary()

# ==============================================================================

forecaster = ForecasterRnn(
    regressor=model,
    levels=levels,
    fit_kwargs={
        "epochs": 2,  # Number of epochs to train the model.
        "batch_size": 2048,  # Batch size to train the model.
    },
)    



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ series_input        │ (None, 24, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 100)       │     40,800 │ series_input[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 36, 100)   │          0 │ lstm_1[0][0]      │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ exog_input          │ (None, 36, 22)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 36, 122)   │          0 │ repeat_vector_1[… │
│ (Concatenate)       │                   │            │ exog_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 36, 128)   │     15,744 │ concatenate[0][0] │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_4  │ (None, 36, 64)    │      8,256 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_5  │ (None, 36, 1)     │         65 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 64,865 (253.38 KB)

 Trainable params: 64,865 (253.38 KB)

 Non-trainable params: 0 (0.00 B)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 20 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
forecaster.fit(series = data[levels], exog = data[exog_features])

Epoch 1/2


c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['series_input', 'exog_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 587ms/step - loss: 0.0738
Epoch 2/2
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 614ms/step - loss: 0.0362


In [30]:
forecaster.create_train_X_y(series = data[levels], exog = data[exog_features])[2]

DatetimeIndex(['2011-01-09 00:00:00', '2011-01-09 01:00:00',
               '2011-01-09 02:00:00', '2011-01-09 03:00:00',
               '2011-01-09 04:00:00', '2011-01-09 05:00:00',
               '2011-01-09 06:00:00', '2011-01-09 07:00:00',
               '2011-01-09 08:00:00', '2011-01-09 09:00:00',
               ...
               '2012-12-29 03:00:00', '2012-12-29 04:00:00',
               '2012-12-29 05:00:00', '2012-12-29 06:00:00',
               '2012-12-29 07:00:00', '2012-12-29 08:00:00',
               '2012-12-29 09:00:00', '2012-12-29 10:00:00',
               '2012-12-29 11:00:00', '2012-12-29 12:00:00'],
              dtype='datetime64[ns]', name='date_time', length=17293, freq='h')


array([[[0.02459016],
        [0.01127049],
        [0.0102459 ],
        ...,
        [0.09528689],
        [0.0307377 ],
        [0.02971311]],

       [[0.01127049],
        [0.0102459 ],
        [0.00307377],
        ...,
        [0.0307377 ],
        [0.02971311],
        [0.0522541 ]],

       [[0.0102459 ],
        [0.00307377],
        [0.        ],
        ...,
        [0.02971311],
        [0.0522541 ],
        [0.05430328]],

       ...,

       [[0.04098361],
        [0.04508197],
        [0.04815574],
        ...,
        [0.10348361],
        [0.0727459 ],
        [0.04713115]],

       [[0.04508197],
        [0.04815574],
        [0.08504098],
        ...,
        [0.0727459 ],
        [0.04713115],
        [0.03586066]],

       [[0.04815574],
        [0.08504098],
        [0.09938525],
        ...,
        [0.04713115],
        [0.03586066],
        [0.04918033]]], shape=(17293, 36, 1))

In [ ]:
forecaster._create_predict_inputs(series = data[levels], exog = data[exog_features])[2]

In [25]:
key = 2

print(forecaster.keras_backend)
print(forecaster.X_train_dim_names_[key])
print(forecaster.y_train_dim_names_[key])
print(forecaster.exog_train_dim_names_[key])
print(forecaster.series_names_in_)
print(forecaster.exog_in_)

tensorflow
['users']
['users']
['month_sin', 'month_cos', 'week_of_year_sin', 'week_of_year_cos', 'week_day_sin', 'week_day_cos', 'hour_day_sin', 'hour_day_cos', 'sunrise_hour_sin', 'sunrise_hour_cos', 'sunset_hour_sin', 'sunset_hour_cos', 'holiday_previous_day', 'holiday_next_day', 'temp_roll_mean_1_day', 'temp_roll_mean_7_day', 'temp_roll_max_1_day', 'temp_roll_min_1_day', 'temp_roll_max_7_day', 'temp_roll_min_7_day', 'temp', 'holiday']
['users']
True


In [26]:
forecaster.in_sample_residuals_

{1: array([[-0.06500238],
        [-0.0714306 ],
        [-0.06677454],
        ...,
        [-0.07362577],
        [-0.07212345],
        [-0.07297853]], shape=(17293, 1)),
 2: array([[-0.07144733],
        [-0.06680337],
        [-0.07019052],
        ...,
        [-0.07211882],
        [-0.07297212],
        [-0.04094816]], shape=(17293, 1)),
 3: array([[-0.06681903],
        [-0.07021936],
        [-0.07019781],
        ...,
        [-0.0729673 ],
        [-0.04094798],
        [-0.03463231]], shape=(17293, 1)),
 4: array([[-0.07023501],
        [-0.07022664],
        [-0.06981433],
        ...,
        [-0.04094875],
        [-0.03463169],
        [-0.00712099]], shape=(17293, 1)),
 5: array([[-0.07024227],
        [-0.06984409],
        [-0.0719807 ],
        ...,
        [-0.03463212],
        [-0.00712109],
        [ 0.0032284 ]], shape=(17293, 1)),
 6: array([[-0.06986038],
        [-0.07201028],
        [-0.0698631 ],
        ...,
        [-0.00712248],
        [ 0.00322837],

In [ ]:

# Backtesting model with exogenous variables on test data 
# # ============================================================================== 
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster_multiseries
 
cv = TimeSeriesFold(steps = 36, initial_train_size = len(data[:end_validation]), refit = False, allow_incomplete_fold=False)
metric, predictions = backtesting_forecaster_multiseries( forecaster = forecaster, series = data[levels], exog = data[exog_features], cv = cv, metric = 'mean_absolute_error' )

metric